# Storage

In [ ]:
## the path to the `mldl2023` folder in your drive
rootMldl = '<your_drive>/mldl2023'

# Initialization

In [ ]:
# packages 1
import shutil
import os
import torch

if not torch.cuda.is_available():
    raise RuntimeError('The model cannot operate without CUDA!')

In [ ]:
# getting the notebook's name
from requests import get
from socket import gethostname, gethostbyname
notebookName = get(f'http://{gethostbyname(gethostname())}:9000/api/sessions').json()[0]['name']

In [ ]:
# mounting the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# changing the root
os.chdir(rootMldl)

In [ ]:
# packages 2
import random
import string
from typing import Any
from typing import List
import numpy as np
from PIL import Image
from torch import from_numpy
from torchvision.datasets import VisionDataset
import datasets.ss_transforms as tr
from utils.stream_metrics import StreamSegMetrics
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from models import deeplabv3, mobilenetv2
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch, os, copy
import tqdm.notebook as tqdm
import gc
from utils.utils import HardNegativeMining, MeanReduction
import math
import json
import csv
from pprint import pprint
from torch import from_numpy
import datetime
import time

# Memory Management

In [ ]:
# packages
!pip install gputil
import prettytable
import psutil
import GPUtil

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7393 sha256=e9bd96bd55d458dd551dd33e1ca15b34fb05e816eb3c745c304a9e89e6f14d14
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built gputil


In [ ]:
# garbage collector
def clearCache():
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
# memory scanner
def printMemoryUsage(title='Memory status'):
    diskStatus = shutil.disk_usage('/content/')
    table = prettytable.PrettyTable(['type', 'available (MB)', 'used (MB)', 'free (MB)'])
    for field in table.field_names:
        table.align[field] = 'l'
    table.add_row([
        'disk',
        round((diskStatus[1]+diskStatus[2])/(1024*1024), 1),
        round((diskStatus[1])/(1024*1024), 1),
        round((diskStatus[2])/(1024*1024), 1)
    ])
    table.add_row([
        'ram',
        round((psutil.virtual_memory().available+psutil.virtual_memory().used)/(1024*1024), 1),
        round(psutil.virtual_memory().used/(1024*1024), 1),
        round(psutil.virtual_memory().available/(1024*1024), 1)
    ])
    for i, gpu in enumerate(GPUtil.getGPUs()):
        table.add_row([
            f'gpu-{i} ram',
            round(gpu.memoryUsed+gpu.memoryFree, 1),
            round(gpu.memoryUsed, 1),
            round(gpu.memoryFree, 1)
        ])
    print(title+':')
    print(table)
    print('')

# Dataset Classes

In [ ]:
class GTAVDataset(VisionDataset):

    @staticmethod
    def get_mapping():
        classes = {
            1: 13, # ego_vehicle : vehicle
            7: 0, # road
            8: 1, # sidewalk
            11: 2, # building
            12: 3, # wall
            13: 4, # fence
            17: 5, # pole
            18: 5, # poleGroup: pole
            19: 6, # traffic light
            20: 7, # traffic sign
            21: 8, # vegetation
            22: 9, # terrain
            23: 10, # sky
            24: 11, # person
            25: 12, # rider
            26: 13, # car : vehicle
            27: 13, # truck : vehicle
            28: 13, # bus : vehicle
            32: 14, # motorcycle
            33: 15, # bicycle
        }
        mapping = np.zeros((256,), dtype=np.int64) + 255
        for i in classes:
            mapping[i] = classes[i]
        return lambda x: from_numpy(mapping[x])

    def __init__(self, root, fileNames, transform=None):
        super().__init__(root=root, transform=transform, target_transform=GTAVDataset.get_mapping())
        self.fileNames = fileNames

    def __getitem__(self, index):
        image = Image.open(self.root+'/images/'+self.fileNames[index]).convert('RGB').resize((1920, 1080), Image.BILINEAR)
        label = Image.open(self.root+'/labels/'+self.fileNames[index]).resize((1920, 1080), Image.BILINEAR)
        if self.transform is not None:
            image, label = self.transform(image, label)
        label = self.target_transform(label)
        return image, label

    def __len__(self):
        return len(self.fileNames)

In [ ]:
class IDDADataset(VisionDataset):

    @staticmethod
    def get_mapping():
        classes = [255, 2, 4, 255, 11, 5, 0, 0, 1, 8, 13, 3, 7, 6, 255, 255, 15, 14, 12, 9, 10]
        mapping = 255*np.ones(256, dtype=np.int64)
        mapping[range(len(classes))] = classes
        return lambda x: from_numpy(mapping[x])

    def __init__(self, root, fileNames, transform=None):
        super().__init__(root=root, transform=transform, target_transform=IDDADataset.get_mapping())
        self.fileNames = fileNames

    def __getitem__(self, index):
        image = Image.open(self.root+'/images/'+self.fileNames[index]+'.jpg').convert('RGB')
        label = Image.open(self.root+'/labels/'+self.fileNames[index]+'.png').convert('L')
        if self.transform is not None:
            image, label = self.transform(image, label)
        label = self.target_transform(label)
        return image, label

    def __len__(self):
        return len(self.fileNames)

# Server class

In [ ]:
class Server:

    def __init__(self,
                 device, model,
                 datasetTrain, datasetTestIdda, datasetTestSame, datasetTestDiff,
                 batchSizeTrain, batchSizeTest,
                 metricClass, num_epochs,
                 scheduler_dict, optimizer_dict,
                 notebookName, pathStorage, lastEpoch, bestEpochIdda, bestMiouIdda, bestEpochSame, bestMiouSame, bestEpochDiff, bestMiouDiff):

        self.device = device
        self.model = model

        self.dataLoaderTrain = DataLoader(datasetTrain, batch_size=batchSizeTrain, shuffle=True, drop_last=True)
        self.dataLoaderTestIdda = DataLoader(datasetTestIdda, batch_size=batchSizeTest, shuffle=False, drop_last=False)
        self.dataLoaderTestSame = DataLoader(datasetTestSame, batch_size=batchSizeTest, shuffle=False, drop_last=False)
        self.dataLoaderTestDiff = DataLoader(datasetTestDiff, batch_size=batchSizeTest, shuffle=False, drop_last=False)

        self.metricClass = metricClass
        self.num_epochs = num_epochs

        # manually programmed cosine annealing scheduler
        self.scheduler = lambda epoch: scheduler_dict['lr_min'] + 0.5*(scheduler_dict['lr_initial']-scheduler_dict['lr_min'])*(1+math.cos(math.pi*epoch/scheduler_dict['T_max']))
        self.optimizer_dict = optimizer_dict

        # STORAGE
        self.notebookName = notebookName                                        # STORAGE
        self.pathStorage = pathStorage                                          # STORAGE
        self.initialEpoch = lastEpoch + 1                                       # STORAGE
        self.bestEpochIdda = bestEpochIdda                                      # STORAGE
        self.bestMiouIdda = bestMiouIdda                                        # STORAGE
        self.bestEpochSame = bestEpochSame                                      # STORAGE
        self.bestMiouSame = bestMiouSame                                        # STORAGE
        self.bestEpochDiff = bestEpochDiff                                      # STORAGE
        self.bestMiouDiff = bestMiouDiff                                        # STORAGE
        # STORAGE

        self.lr = self.scheduler(self.initialEpoch)

    def step(self):
        self.epoch += 1
        self.lr = self.scheduler(self.epoch)

    def select_clients(self):
        num_clients = min(self.max_num_clients_per_epoch, len(self.clients))
        return random.sample(self.clients, num_clients)

    def train(self):
        self.epoch = self.initialEpoch
        for epoch in range(self.initialEpoch, self.num_epochs):
            print(f'--------------------- epoch {epoch:03d} out of {self.num_epochs:03d} ---------------------')

            start = time.perf_counter()
            lossTrain, miouTrain = self.train_epoch()
            durationTrain = int(time.perf_counter() - start)
            print(f'-- loss (training): {lossTrain:.5f} -- miou (training): {100*miouTrain:.3f}% -- duration (training): {durationTrain}s')

            start = time.perf_counter()
            miouTestIdda = self.test(self.dataLoaderTestIdda)
            durationTestIdda = int(time.perf_counter() - start)
            print(f'-- miou (test - idda): {100*miouTestIdda:.3f}% -- duration (test - idda): {durationTestIdda}s')

            start = time.perf_counter()
            miouTestSame = self.test(self.dataLoaderTestSame)
            durationTestSame = int(time.perf_counter() - start)
            print(f'-- miou (test - same): {100*miouTestSame:.3f}% -- duration (test - same): {durationTestSame}s')

            start = time.perf_counter()
            miouTestDiff = self.test(self.dataLoaderTestDiff)
            durationTestDiff = int(time.perf_counter() - start)
            print(f'-- miou (test - diff): {100*miouTestDiff:.3f}% -- duration (test - diff): {durationTestDiff}s')

            ## STORAGE
            print('-- storing data')
            with open(self.pathStorage+'/'+'metrics.csv', 'a') as file:
                csv_writer = csv.writer(file)
                csv_writer.writerow([epoch, lossTrain, miouTrain, miouTestIdda, miouTestSame, miouTestDiff, durationTrain, durationTestIdda, durationTestSame, durationTestDiff])
                file.close()

            torch.save(self.model.state_dict(), self.pathStorage+f'/model-main.pth')
            models = ['main']
            if miouIdda > self.bestMiouIdda:
                self.bestEpochIdda = epoch
                self.bestMiouIdda = miouIdda
                torch.save(self.model.state_dict(), self.pathStorage+f'/model-bestIdda.pth')
                models.append('bestIdda')
            if miouSame > self.bestMiouSame:
                self.bestEpochSame = epoch
                self.bestMiouSame = miouSame
                torch.save(self.model.state_dict(), self.pathStorage+f'/model-bestSame.pth')
                models.append('bestSame')
            if miouDiff > self.bestMiouDiff:
                self.bestEpochDiff = epoch
                self.bestMiouDiff = miouDiff
                torch.save(self.model.state_dict(), self.pathStorage+f'/model-bestDiff.pth')
                models.append('bestDiff')

            with open(self.pathStorage+'/'+'log.txt', 'a') as file:
                file.write(f'-- models were overwritten on '+datetime.datetime.now().strftime('%Y-%m-%d at %H:%M:%S')+f' by {self.notebookName}\n')
                file.write(f'   models: [{", ".join(models)}]\n')
                file.write(f'   last successful epoch: {epoch}\n')
                file.write(f'   best record of idda: [{self.bestEpochIdda}, {self.bestMiouIdda}]\n')
                file.write(f'   best record of same: [{self.bestEpochSame}, {self.bestMiouSame}]\n')
                file.write(f'   best record of diff: [{self.bestEpochDiff}, {self.bestMiouDiff}]\n')
                file.close()
            ## STORAGE

    def train_epoch(self):

        # training the model
        optimizer = torch.optim.SGD(self.model.parameters(), lr=self.lr, **self.optimizer_dict)
        criterion = nn.CrossEntropyLoss(ignore_index=255)
        clearCache()
        self.model.train()

        lossCum = 0.0
        metric = self.metricClass(n_classes=21, name='miou')
        pbar = tqdm.tqdm(
            total = len(self.dataLoaderTrain),
            desc = 'training'
        )
        for batch, (images, labels) in enumerate(self.dataLoaderTrain):
            pbar.set_postfix({
                'batch': f'{batch+1}/{len(self.dataLoaderTrain)}'
            })
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = self.model(images)['out']
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            lossCum = (batch*lossCum + loss.item())/(batch + 1)
            _, prediction = outputs.max(dim=1)
            metric.update(labels.cpu().numpy(), prediction.cpu().numpy())
            pbar.update(1)

        # closing the progress bar
        pbar.close()

        # stepping the scheduler
        self.step()

        # returning the results
        miouCum = metric.get_results()['Mean IoU']
        return lossCum, miouCum

    def test(self, dataLoader):
        clearCache()
        metric = self.metricClass(n_classes=21, name='miou')
        with torch.no_grad():
            self.model.eval()
            pbar = tqdm.tqdm(total=len(dataLoader), desc='testing')
            for batch, (images, labels) in enumerate(dataLoader):
                pbar.set_postfix({
                    'batch': f'{batch+1}/{len(dataLoader)}'
                })
                images = images.to(self.device)
                labels = labels.to(self.device)
                outputs = self.model(images)['out']
                _, prediction = outputs.max(dim=1)
                metric.update(labels.cpu().numpy(), prediction.cpu().numpy())
                pbar.update(1)
        pbar.close()
        miouCum = metric.get_results()['Mean IoU']
        return miouCum

# FDA Class

In [ ]:
"""
this class can be used just like a pytorch transformer,
just initialize it with the root to the styles and the size of the middle
square wanted and put this transformer in the "tr.Compose()"
"""


class FDA(object):

    def __init__(self, root, size=2):
        self.size = size
        self.root = root
        self.clientstyles = dict()
        for fileName in os.listdir(root):
            base, extension = os.path.splitext(fileName)
            if extension == '.json':
                with open(root+'/'+fileName) as f:
                    clientstyle = json.load(f)
                    style = np.array(clientstyle[base])
                    rows, cols, channels = style.shape
                    row_start = rows // 2 - int(self.size/2)
                    row_end = rows // 2 + int(self.size/2)
                    col_start = cols // 2 - int(self.size/2)
                    col_end = cols // 2 + int(self.size/2)

                    # save only the part that will be used of the clients' styles
                    style = style[row_start:row_end, col_start:col_end, :]
                    self.clientstyles[base] = style
                    f.close

    def __call__(self, img, lbl=None):
        #convert the image to numpy
        imgnp = np.array(img)

        #get a random client
        name =  random.choice(list(self.clientstyles.keys()))

        # convert the list of lists into numpy array
        style = self.clientstyles[name]

        #fft of the gta image, on the height and width
        imgfft = np.fft.fftshift(np.fft.fft2(imgnp, axes=(0, 1)), axes=(0, 1))

        # get the shape of the image
        rows, cols, channels = imgnp.shape

        # define the coordinates of the middle part
        row_start = rows // 2 - int(self.size/2)
        row_end = rows // 2 + int(self.size/2)
        col_start = cols // 2 - int(self.size/2)
        col_end = cols // 2 + int(self.size/2)

        # get amplitude and phase of the fft
        imgfftamplitude = np.abs(imgfft)
        phase = np.angle(imgfft)

        #exchange the middle part with the client style
        imgfftamplitude[row_start:row_end, col_start:col_end, :] = style

        # inverse Fourier Transform using the modified amplitude (keeping the phase intact)
        # different sources showed that i needed to do np.real or np.abs
        # i tested both, and what looked like worked better was np.abs
        # so i left it with np.abs
        mod = np.abs(np.fft.ifft2(np.fft.ifftshift(imgfftamplitude * np.exp(1j * phase), axes=(0,1)), axes=(0, 1)))

        # clip and convert the result back to the original image type (e.g., uint8)
        mod = np.clip(mod, 0, 255)
        mod = mod.astype(np.uint8)

        # Convert the modified image array back to PIL image
        mod = Image.fromarray(mod)

        if lbl is None:
            return mod
        else:
            return mod, lbl

# Different Things

In [ ]:
# params class
class Params:
    def __init__(self, **args):
        for key, value in args.items():
            setattr(self, key, value)

In [ ]:
# names reader
def getFileNames(root, containerName):
    fileNames = []
    with open(os.path.join(root, containerName), 'r') as file:
        for line in file.read().splitlines():
            fileNames.append(line)
    return fileNames

In [ ]:
# main function
def main(pathStorage, params=None, config=None):
    print('path: '+pathStorage)

    # storage
    if os.path.exists(pathStorage):
        with open(pathStorage+'/log.txt') as file:
            lines = file.readlines()
            lastEpochLine = lines[-4]
            recordIddaLine = lines[-3]
            recordSameLine = lines[-2]
            recordDiffLine = lines[-1]
            lastEpoch = lastEpochLine[lastEpochLine.find(':')+2:-1]
            bestEpochIdda = int(recordIddaLine[recordIddaLine.find(':')+3:recordIddaLine.find(',')])
            bestMiouIdda = float(recordIddaLine[recordIddaLine.find(',')+2:-2])
            bestEpochSame = int(recordSameLine[recordSameLine.find(':')+3:recordSameLine.find(',')])
            bestMiouSame = float(recordSameLine[recordSameLine.find(',')+2:-2])
            bestEpochDiff = int(recordDiffLine[recordDiffLine.find(':')+3:recordDiffLine.find(',')])
            bestMiouDiff = float(recordDiffLine[recordDiffLine.find(',')+2:-2])
            file.close()

        with open(pathStorage+'/params.json') as file:
            params = Params(**json.load(file))
            file.close()

        clearCache()
        model = deeplabv3.deeplabv3_mobilenetv2().to(params.device)
        model.load_state_dict(torch.load(pathStorage+'/'+f'model-main.pth'))

        with open(pathStorage+'/log.txt', 'a') as file:
            file.write(f'-- models were loaded on '+datetime.datetime.now().strftime('%Y-%m-%d at %H:%M:%S')+f' by {notebookName}\n')
            file.write(f'   models: [main]\n')
            file.write(f'   last successful epoch: {lastEpoch}\n')
            file.write(recordIddaLine)
            file.write(recordSameLine)
            file.write(recordDiffLine)
            file.close()
        lastEpoch = -1 if lastEpoch == 'n/a' else int(lastEpoch)
        if (lastEpoch == params.num_epochs - 1):
            print('-- this config is finished!')
            return

    else:
        for key, value in config.items():
            if hasattr(params, key):
                setattr(params, key, value)
            else:
                raise RuntimeError(f'Make sure each key of `config` is already an attribute of `params`! The key `{key}` does not exist!')

        os.makedirs(pathStorage, exist_ok=True)

        with open(pathStorage+'/params.json', 'w') as file:
            json.dump(params.__dict__, file, indent=4)
            file.close()

        with open(pathStorage+'/config.json', 'w') as file:
            json.dump(dict(config), file, indent=4)
            file.close()

        with open(pathStorage+'/metrics.csv', 'w', newline='') as file:
            csv_writer = csv.writer(file)
            csv_writer.writerow(['epoch', 'lossTrain', 'miouTrain', 'miouTestIdda', 'miouTestSame', 'miouTestDiff', 'durationTrain', 'durationTestIdda', 'durationTestSame', 'durationTestDiff'])
            file.close()

        clearCache()
        model = deeplabv3.deeplabv3_mobilenetv2().to(params.device)

        torch.save(model.state_dict(), pathStorage+'/model-main.pth')
        torch.save(model.state_dict(), pathStorage+'/model-bestIdda.pth')
        torch.save(model.state_dict(), pathStorage+'/model-bestSame.pth')
        torch.save(model.state_dict(), pathStorage+'/model-bestDiff.pth')
        with open(pathStorage+'/log.txt', 'w') as file:
            file.write('-- models were created on '+datetime.datetime.now().strftime('%Y-%m-%d at %H:%M:%S')+f' by {notebookName}\n')
            file.write('   models: [main, bestIdda, bestSame, bestDiff]\n')
            file.write('   last successful epoch: n/a\n')
            file.write('   best record of idda: [0, 0.0]\n')
            file.write('   best record of same: [0, 0.0]\n')
            file.write('   best record of diff: [0, 0.0]\n')
            file.close()
        lastEpoch = -1
        bestEpochIdda = 0
        bestMiouIdda = 0.0
        bestEpochSame = 0
        bestMiouSame = 0.0
        bestEpochDiff = 0
        bestMiouDiff = 0.0

    # transformers
    transformsTrain = tr.Compose([
        FDA(root=params.rootIdda+'/styles', size=params.transformer_fdaSize),
        tr.RandomResizedCrop(size=tuple(params.transformer_imageSize),scale=params.transformer_scale),
        tr.ColorJitter(*params.transformer_jitter),
        tr.RandomHorizontalFlip(),
        tr.ToTensor(),
        tr.Normalize(tuple(params.transformer_means), tuple(params.transformer_stds))
    ])
    transformsTest = tr.Compose([
        tr.ToTensor(),
        tr.Normalize(tuple(params.transformer_means), tuple(params.transformer_stds))
    ])

    # server
    server = Server(
        device = params.device,
        model = model,
        datasetTrain = GTAVDataset(
            root = params.rootGtav,
            fileNames = getFileNames(params.rootGtav, 'train.txt'),
            transform = transformsTrain
        ),
        datasetTestIdda = IDDADataset(
            root = params.rootIdda,
            fileNames = getFileNames(params.rootIdda, 'train.txt'),
            transform = transformsTest
        ),
        datasetTestSame = IDDADataset(
            root = params.rootIdda,
            fileNames = getFileNames(params.rootIdda, 'test_same_dom.txt'),
            transform = transformsTest
        ),
        datasetTestDiff = IDDADataset(
            root = params.rootIdda,
            fileNames = getFileNames(params.rootIdda, 'test_diff_dom.txt'),
            transform = transformsTest
        ),
        batchSizeTrain = params.batchSizeTrain,
        batchSizeTest = params.batchSizeTest,
        metricClass = StreamSegMetrics,
        num_epochs = params.num_epochs,
        scheduler_dict = {
            'lr_initial': params.scheduler_lr_initial,
            'lr_min':  params.scheduler_lr_min,
            'T_max': params.scheduler_T_max
        },
        optimizer_dict = {
            'momentum': params.optimizer_momentum,
            'weight_decay': params.optimizer_weight_decay
        },
        notebookName = notebookName,                                            # STORAGE
        pathStorage = pathStorage,                                              # STORAGE
        lastEpoch = lastEpoch,                                                  # STORAGE
        bestEpochIdda = bestEpochIdda,                                          # STORAGE
        bestMiouIdda = bestMiouIdda,                                            # STORAGE
        bestEpochSame = bestEpochSame,                                          # STORAGE
        bestMiouSame = bestMiouSame,                                            # STORAGE
        bestEpochDiff = bestEpochDiff,                                          # STORAGE
        bestMiouDiff = bestMiouDiff                                             # STORAGE
    )

    # train federated learning
    server.train()

# Server Driver

In [ ]:
# params
## the intitial parameters (default config)
## if there's a config value for a parameter, it'll be overwritten
params = Params(
    device = 'cuda:0',
    rootGtav = '/content/data/gtav',
    rootIdda = '/content/data/idda',
    rootStorage = rootMldl+'/storage/step3/part4',
    batchSizeTrain = 3,
    batchSizeTest = 3,
    transformer_fdaSize = 2,
    transformer_imageSize = [1080, 1920],
    transformer_scale = [0.25, 1],
    transformer_jitter = [0.4, 0.4, 0.5, 0.1],
    transformer_means = [0.320888, 0.292300, 0.288562],
    transformer_stds  = [0.250606, 0.248234, 0.253670],
    num_epochs = 100,                                                           # server
    scheduler_lr_initial = 0.1,                                                 # server (scheduler parameter)
    scheduler_lr_min = 0,                                                       # server (scheduler parameter)
    scheduler_T_max = 300,                                                      # server (scheduler parameter)
    optimizer_momentum = 0.65,                                                  # client (optimzer parameter)
    optimizer_weight_decay = 0.0005                                             # client (optimzer parameter)
)

In [ ]:
# copies
print('Copying GTAV dataset ...')
shutil.copytree(rootMldl+'/data/GTA5', params.rootGtav, dirs_exist_ok=True)
print('Copying IDDA data ...')
shutil.copytree(rootMldl+'/data/idda', params.rootIdda, dirs_exist_ok=True)
print('Copying styles ...')
shutil.copytree(rootMldl+'/data/idda/styles', params.rootIdda+'/styles', dirs_exist_ok=True)

In [ ]:
# configs
## make sure each key of `config` is already an attribute of `params`
config = {
    'transformer_fdaSize': 4,
}

# storage
folderName = []
for key, value in config.items():
    args = key.split('_')
    folderName.append((''.join([arg[:2].capitalize() for arg in args]))+'='+str(value))
folderName = ','.join(folderName)

# driver
pathStorage = params.rootStorage+'/'+folderName
if os.path.exists(params.rootStorage) and (folderName in os.listdir(params.rootStorage)):
    main(pathStorage)
else:
    main(
        params = params,
        config = config,
        pathStorage = params.rootStorage+'/'+folderName
    )